In [31]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from fuel import GraphAdjacencyMatrixExtractor

In [40]:
def load_ground_truth():
    """
    Load ground truth adjacency matrices from CSV file with new column structure.
    """
    # Define the expected columns and header row
    column_names= ['graph_id','num_nodes'] + [f'a{i}' for i in range(36)]

    # Read the CSV file
    df = pd.read_csv('adjacency_matrices.csv', header=0)

    ground_truth = {}
    for idx, row in df.iterrows():
        try:
            graph_id = int(row['graph_id'])
            num_nodes = int(row['num_nodes'])
            if num_nodes == 0:
                continue
            # Extract adjacency matrix values (a0 ... a35)
            matrix_values = [int(row[f'a{i}']) for i in range(num_nodes * num_nodes)]
            matrix = np.array(matrix_values, dtype=int).reshape(num_nodes, num_nodes)
            ground_truth[graph_id] = matrix
        except Exception:
            continue
    return ground_truth


In [41]:

def calculate_metrics(predicted, ground_truth):
    """
    Calculate accuracy metrics by comparing predicted adjacency matrices with ground truth
    """
    all_true = []
    all_pred = []

    # Ensure keys are int for matching
    pred_keys = set(int(k) for k in predicted.keys())
    gt_keys = set(int(k) for k in ground_truth.keys())
    common_keys = pred_keys & gt_keys

    for graph_id in sorted(common_keys):
        gt_matrix = ground_truth[graph_id]
        pred_matrix = predicted[graph_id]
        # If matrices have different shapes, skip
        if gt_matrix.shape != pred_matrix.shape:
            continue
        all_true.extend(gt_matrix.flatten().astype(int).tolist())
        all_pred.extend(pred_matrix.flatten().astype(int).tolist())

    # Calculate metrics
    if len(all_true) > 0 and len(all_pred) > 0:
        accuracy = accuracy_score(all_true, all_pred)
        precision = precision_score(all_true, all_pred, zero_division=0)
        recall = recall_score(all_true, all_pred, zero_division=0)
        f1 = f1_score(all_true, all_pred, zero_division=0)
    else:
        accuracy = 0
        precision = 0
        recall = 0
        f1 = 0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [42]:

def main(image_dir='graphs_images'):
    """
    Process all graph images in the directory and save results to CSV
    """
    extractor = GraphAdjacencyMatrixExtractor()
    predicted = {}
    results = []

    # Use the new column names and header row
    column_names = ['graph_id','num_nodes'] + [f'a{i}' for i in range(36)]
    results = []

    max_cols = 38  # 2 for id and num_nodes, 36 for a0..a35

    # Get all image files
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    for image_file in image_files:
        # Extract graph_id
        graph_id = int(image_file.split('.')[0])
        image_path = os.path.join(image_dir, image_file)

        # Extract adjacency matrix
        adjacency_matrix, labels = extractor.extract_adjacency_matrix(image_path)

        if len(adjacency_matrix) > 0:
            # Order the adjacency matrix based on node labels
            ordered_matrix, ordered_labels = extractor.order_adjacency_matrix(adjacency_matrix, labels)
            predicted[graph_id] = ordered_matrix
            flat_values = ordered_matrix.flatten().tolist()
            # Pad with zeros to always have 36 values (for up to 6x6 matrices)
            flat_padded = flat_values + [0] * (36 - len(flat_values))
            row = [graph_id, len(ordered_matrix)] + flat_padded
            results.append(row)

    # Create DataFrame with new column names
    results_df = pd.DataFrame(results, columns=column_names)
    # Separate the first row
    first_row = results_df.iloc[[0]]
    # Sort the rest of the DataFrame
    rest_sorted = results_df.iloc[1:].sort_values(by='graph_id')
    # Concatenate the first row back on top
    results_df = pd.concat([first_row, rest_sorted], ignore_index=True)
    # Save to CSV
    results_df.to_csv('output.csv', index=False)

    # Calculate accuracy
    ground_truth = load_ground_truth()
    metrics = calculate_metrics(predicted, ground_truth)

    # Display metrics
    print("\n===== Adjacency Matrix Extraction Metrics =====")
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1 Score:  {metrics['f1']:.4f}")
    print("===============================================\n")

    # Save metrics to a file
    with open('accuracy_metrics.txt', 'w') as f:
        for metric, value in metrics.items():
            f.write(f"{metric}: {value:.4f}\n")

    return metrics

if __name__ == "__main__":
    metrics = main()


===== Adjacency Matrix Extraction Metrics =====
Accuracy:  0.9715
Precision: 0.9253
Recall:    0.9876
F1 Score:  0.9554



In [43]:
import pandas as pd
import numpy as np
from fuel import solve_minimum_fuel_optimized  # Make sure this is imported correctly

def load_matrices(csv_path):
    df = pd.read_csv(csv_path)

    matrices = {}
    try:
        for _, row in df.iterrows():

            graph_id = row.iloc[0]
            if graph_id == 'graph_id':
                continue
            else:
                graph_id = int(graph_id)
            num_nodes = int(row.iloc[1])
            if num_nodes == 0:
                continue
            matrix_values = [int(row[f'a{i}']) for i in range(num_nodes * num_nodes)]
            matrix = np.array(matrix_values, dtype=int).reshape(num_nodes, num_nodes)
            matrices[graph_id] = matrix
        return matrices
    except Exception:
        return matrices

def compute_fuel_for_all(matrices):
    fuels = {}
    for graph_id, matrix in matrices.items():
        # By default, start at node 0, end at last node
        start_idx = 0
        end_idx = matrix.shape[0] - 1
        try:
            fuel = solve_minimum_fuel_optimized(matrix, start_idx, end_idx)
        except Exception:
            fuel = -1  # Or np.inf if unreachable
        fuels[graph_id] = fuel
    return fuels

def practical_accuracy(pred_fuels, gt_fuels):
    common_ids = set(pred_fuels.keys()) & set(gt_fuels.keys())
    correct = sum(pred_fuels[i] == gt_fuels[i] for i in common_ids)
    total = len(common_ids)
    return correct / total if total > 0 else 0

if __name__ == "__main__":
    pred_matrices = load_matrices('output.csv')
    gt_matrices = load_matrices('adjacency_matrices.csv')

    pred_fuels = compute_fuel_for_all(pred_matrices)
    gt_fuels = compute_fuel_for_all(gt_matrices)

    acc = practical_accuracy(pred_fuels, gt_fuels)
    print(f"Practical accuracy (fuel match): {acc:.4f}")

Practical accuracy (fuel match): 0.9260
